## Cleaning the Season Master CSV and Creating Season Modeling CSV 

### Importing required libraries

In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
from sklearn import datasets
from sklearn.model_selection import train_test_split  
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import copy
from sklearn.linear_model import LogisticRegression
import os
import difflib
import math
import re
import time

### Define the year to get the modeling file and run all cells
<b><u> Note: </u></b> The paths for input files (the master_df, pitcher_era) and output file (modeling csv) can be changed here as well

In [229]:
year_to_create_modeling_csv = "2019"
path_to_save_modeling_csv_for_year = '/Users/carlylagrotta/Dropbox/Columbia/Fall_2020_Semester/Data_Science/Data_Science_For_Mechanical_Systems/ModelingCSVs/modeling_'+str(year_to_create_modeling_csv)+'.csv'
path_for_master_csv_for_year = '/Users/carlylagrotta/Dropbox/Columbia/Fall_2020_Semester/Data_Science/Data_Science_For_Mechanical_Systems/Master_Data/master_df_'+str(year_to_create_modeling_csv)+'.csv'
path_for_picther_era_for_year = '/Users/carlylagrotta/Dropbox/Columbia/Fall_2020_Semester/Data_Science/Data_Science_For_Mechanical_Systems/ERA/PitcherEra_'+str(year_to_create_modeling_csv)+'.csv'

### Creating necessary feature mapping dictionaries

Dictionaries for:
- Converting stadium name to turf or grass (ground condition)
- Converting stadium name to open or closed dome type stadium
- Converting Pitcher and Batter handedness to a numeric variable
- Converting difference of Pitcher and Batter handedness to a numeric variable


In [226]:
turff_or_grass = {'ARI':1,
 'ATL':0,
 'BAL':0,
 'BOS':0,
 'CHC':0,
 'CIN':0,
 'CLE':0,
 'COL':0,
 'CWS':0,
 'DET':0,
 'HOU':0,
 'KC':0,
 'LAA':0,
 'LAD':0,
 'MIA':1,
 'MIL':0,
 'MIN':0,
 'NYM':0,
 'NYY':0,
 'OAK':0,
 'PHI':0,
 'PIT':0,
 'SD':0,
 'SEA':0,
 'SF':0,
 'STL':0,
 'TB':1,
 'TEX':1,
 'TOR':1,
 'WSH':0}

In [227]:
dome = {'ARI':1,
 'ATL':0,
 'BAL':0,
 'BOS':0,
 'CHC':0,
 'CIN':0,
 'CLE':0,
 'COL':0,
 'CWS':0,
 'DET':0,
 'HOU':1,
 'KC':0,
 'LAA':0,
 'LAD':0,
 'MIA':1,
 'MIL':1,
 'MIN':0,
 'NYM':0,
 'NYY':0,
 'OAK':0,
 'PHI':0,
 'PIT':0,
 'SD':0,
 'SEA':1,
 'SF':0,
 'STL':0,
 'TB':2,
 'TEX':1,
 'TOR':1,
 'WSH':0}

In [228]:
pitcher_hand = {'R':0,'L':1,'S':3}
batter_hand = {' R ': 0 ,' L ':1,' S ':3}

In [229]:
hit = {True:1, 
       False:0}

In [230]:
compare_batter_pitcher_hand = {0:0,
                               1:1,
                              -1:1,
                               3:1,
                               2:1,
                              -3:0,
                               -2:0}

### Function to map the dictionaries defined above 

In [231]:
def CreateNumericalVariable(df,column_name):    
    variable_list = list(set(df[column_name].to_list()))
    numeric_list = []
    for value in range(len(variable_list)):
        numeric_list.append(value)
    mapping = dict(zip(variable_list,numeric_list))
    return mapping

### Function to apply numeric mapping to specficed dataframe columns

In [232]:
def ChangingAColumnToDummyVariable(df,column_name,mapping_dict,type_col = "float"):
    df = df.replace({column_name: mapping_dict})
    df[column_name] = pd.to_numeric(df[column_name], downcast=type_col)
    return df 

In [233]:
#def FillDataNumberOfHits(df):
#    df['Number_Of_Hits'] = df['Number_Of_Hits'].replace(np.nan, 0)
#    df = df.replace('False', np.nan)
#    return df

### Function to fill in empty plate appearnce cells in dataframe
 <b> <u> Note:</u> </b> Estimation for a player's missing plate appearance was determined by his position in the batting line-up <br>
<i> <b>Source: </b></i> https://fantasy.fangraphs.com/buying-generic-plate-appearances-by-lineup-spot/

In [234]:
def FillDataPlateAppearances(df,dict_avg_appr ={1:4.65,
            2:4.55,
            3:4.43,
            4: 4.33,
            5:4.24,
             6:4.13,
             7:4.01,
             8:3.90,
             9:3.77}):
    df['Number_Of_Plate_Appearances'] = df['Number_Of_Plate_Appearances'].fillna(dict_avg_appr)
    return df

### Functions to copy the specified dataframe and drop unwanted column from any specified dataframe

In [235]:
def CopyOriginalDataFrame(df):
    df_clean = copy.deepcopy(df)
    return df_clean

def DropUnwantedColumn(df,names=[]):
    df = df.drop(names,axis=1)
    return df

### Function to fill empty batting average cells in dataframe

 <u> <b> Note: </b></u> Uses the ffill pandas function to assign a player's missing batting average with that of the most recent previous value available for the same player

In [236]:
def FillDataBattingAverage(df):
    df_null_batting_average = df[df['Batting_Average'].isna()]
    for index, row in df_null_batting_average.iterrows():   
        batter_name = row['Batter']    
        df[(df["Batter"]==batter_name)]["Batting_Average"].ffill(inplace=True)

    df["Batting_Average"] = pd.to_numeric(df["Batting_Average"], downcast="float")
    return df



### Importing input files
Reading the Master CSV File for the year (does not contain ERA data)

In [237]:
df_master = pd.read_csv(path_for_master_csv_for_year)
df_master = df_master.drop(['Unnamed: 0'],axis=1)
df_master = df_master.drop_duplicates()

Reading the ERA CSV File for the year

In [238]:
df_era = pd.read_csv(path_for_picther_era_for_year)
df_era = df_era.rename(columns={"Pitching": "Pitcher"})
df_era = df_era.drop(['Unnamed: 0'],axis=1)

### Merging the two CSV dataframes
<b> <u> Note:</u> </b> When merging the two CSV dataframes any date that exists only in one of the CSV dataframe is removed from the combined dataframe


In [239]:
df_cleaning = df_master.merge(df_era,on=['Date','Pitcher'])
#df_cleaning = CopyOriginalDataFrame(df_merged)

Running functions to change categorical variables to numerical mapping in the following order:
1. Batter
2. Pitcher
3. Batter handedness
4. Pitcher handedness
5. Stadium turf
6. Stadium dome type
7. Batter Home


In [240]:
df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Batter',CreateNumericalVariable(df_cleaning,'Batter'),type_col = "integer")

df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Pitcher',CreateNumericalVariable(df_cleaning,'Pitcher'),type_col = "integer")

df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Batter_Handedness',batter_hand,type_col = "integer")

df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Pitcher_Handedness',pitcher_hand,type_col = "integer")

df_cleaning['Stadium_Turf'] = df_cleaning['Stadium']
df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Stadium_Turf',turff_or_grass,type_col = "integer")

df_cleaning['Stadium_Dome'] = df_cleaning['Stadium']
df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Stadium_Dome',dome,type_col = "integer")

df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Batter_Home',CreateNumericalVariable(df_cleaning,'Batter_Home'),type_col = "integer")

 Creating a new column in the cleaning dataframe to show the difference between pitcher and batter handedness and mapping it to a variable

In [241]:
df_cleaning['Difference_Batter_And_Pitcher'] = df_cleaning['Batter_Handedness'] - df_cleaning['Pitcher_Handedness']
df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Difference_Batter_And_Pitcher',compare_batter_pitcher_hand,type_col = "integer")

Running the functions to fill in missing plate apperances and missing batting averages <br>
<b><u> Note: </u></b> The missing batting averages are denoted by "False" in the dataframe, which is replaced by NaN variable for easy fill in.

In [242]:
df_cleaning = FillDataPlateAppearances(df_cleaning)

df_cleaning = df_cleaning.replace("False", np.nan)
df_cleaning = FillDataBattingAverage(df_cleaning)

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/Users/carlylagrotta/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

At this point in the dataframe, all the empty cells (or ones containing Nan) where the values could be estimated are filled. <br>
So any remaining Nan should be due to "Number_Of_Hits" column or some other cases which cannot be realistically estimated for a future game <br>
Thus, any rows containing Nan is dropped.

In [243]:
df_cleaning = df_cleaning.dropna() 

Creating the following:
 - "Hit" column based on the "Number_Of_Hits" column and then converting "Hit" column categorical boolean variable to numerical mapping defined previously. 
 - "No_Hit" column based on "Hit" column 

In [244]:
df_cleaning['Number_Of_Hits'] = pd.to_numeric(df_cleaning['Number_Of_Hits'], downcast='integer')
df_cleaning['Hit'] = (df_cleaning['Number_Of_Hits'] > 0)
df_cleaning = ChangingAColumnToDummyVariable(df_cleaning,'Hit',hit,type_col = "integer")

df_cleaning['No_Hit'] = 1 - df_cleaning['Hit']

Converting "Number_Of_Plate_Appeerances" column to integer type and "Date" column to "Date" type

Finally, creating "Hit_Prob" column based on "Number_Of_Hits" and "Number_Of_Plate_Appearances"

In [245]:
df_cleaning['Number_Of_Plate_Appearances'] = pd.to_numeric(df_cleaning['Number_Of_Plate_Appearances'], downcast='integer')
df_cleaning['DateTime'] = pd.to_datetime(df_cleaning['Date'], format='%Y-%m-%d')

df_cleaning['Hit_Prob'] = df_cleaning['Number_Of_Hits']/df_cleaning['Number_Of_Plate_Appearances']

### Export the output Modeling Csv file

In [246]:
df_cleaning.to_csv(path_to_save_modeling_csv_for_year)